In [1]:
pip install matplotlib

  Using cached matplotlib-3.4.2-cp38-cp38-manylinux1_x86_64.whl (10.3 MB)
  Using cached Pillow-8.3.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (3.0 MB)
  Using cached kiwisolver-1.3.1-cp38-cp38-manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached regex-2021.7.6-cp38-cp38-manylinux2014_x86_64.whl (737 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer 
import re
from datetime import date, datetime, timedelta
from nltk.probability import FreqDist

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [396]:
df=pd.read_csv('full_dataset.csv')
df.head(1)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,index,title,media,date,datetime,desc,link,img
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,Seattle police release video of fatal shooting...,The Seattle Times,01/01/2019,NaN,"Faletogo died of a gunshot wound to the head, ...",https://www.seattletimes.com/seattle-news/crim...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [397]:
#Convert string values in date column to datetime values 
#df['date']=df['date'].apply(lambda x: datetime.strptime(str(x), '%m/%d/%Y'))
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [398]:
# Drop index and img columns and unneeded columns
df = df.drop(['index', 'img', 'datetime','Unnamed: 0','Unnamed: 0.1',
              'Unnamed: 0.1.1','Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1',
              'Unnamed: 0.1.1.1.1.1','Unnamed: 0.1.1.1.1.1.1','index'], axis=1)

In [399]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57051 entries, 0 to 57050
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   title   57051 non-null  object        
 1   media   55081 non-null  object        
 2   date    56761 non-null  datetime64[ns]
 3   desc    57051 non-null  object        
 4   link    57051 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 2.2+ MB


In [400]:
#Fill in Missing values in 'media'
df['media']=df.media.fillna('Others')

In [401]:
#Number of unique news sources
len(df.media.unique())

570

In [402]:
#Top 10 news sources
top_10=df.media.value_counts().head(10)
top_10

The Seattle Times           11294
KIRO-TV                      6641
KING 5                       3002
MyNorthwest.com              2634
Others                       1970
KOMO News                    1706
The Spokesman-Review         1651
KOMO                         1550
KUOW                         1466
CHS Capitol Hill Seattle     1332
Name: media, dtype: int64

In [403]:
#Create Local vs national media dictionary 0-local, 1 = national "Others" =0
media_dict = {
    'The Seattle Times': 0, 
       'West Seattle Blog...':0, 'Others':0 , 'CHS Capitol Hill Seattle':0,
        'Westside Seattle':0, 'PubliCola':0, 'South Seattle Emerald':0,
       'KIRO-TV':0,  'KOMO':0,  'Seattle PI':0,
       'Real Change':0, 'KING 5':0,  'The Stranger':0,
       'Council Connection':0, 'Q13 FOX':0,
       'MyNorthwest.com':0,
       'KOMO News':0,
       'Seattle Met':0,   'Seattle Weekly':0,
    'Northwest Asian Weekly':0,  'West Seattle Blog…': 0,
    'Seattle Spectator': 0, 'Washington Times': 0,   
    'Seattle Magazine |':0, 'Seattle Met':0,
       'Seattle PI':0, 'Seattle Refined':0, 'Seattle Spectator':0,'South Seattle Emerald':0,
       'Seattle Times Interactives Projects':0, 'Seattle Weekly':0,'The Seattle Globalist':0, 'The Seattle Medium':0,
     'West Seattle Blog...':0, 'West Seattle Blog…':0, 'Westside Seattle':0, 'Real Change':0, 
    'International Examiner':0,'dailyuw.com':0


}

In [404]:
for x in df['media']:
    if x not in media_dict:
        media_dict[x]=1
    
df['Local']=[media_dict[x] for x in df['media']]

df.head()

,title,media,date,desc,link,Local
0,Seattle police release video of fatal shooting...,The Seattle Times,2019-01-01,"Faletogo died of a gunshot wound to the head, ...",https://www.seattletimes.com/seattle-news/crim...,0
1,Fatal Seattle police shooting will test new ac...,Crosscut,2019-01-01,Faletogo was stopped by police officers while ...,https://crosscut.com/2019/01/fatal-seattle-pol...,1
2,Seattle Police release body cam footage in fat...,MyNorthwest.com,2019-01-01,The Seattle Police Department released the bod...,https://mynorthwest.com/1232749/seattle-police...,0
3,Former Seattle police chief Jim Pugel will run...,The Seattle Times,2019-01-01,More than two dozen candidates are running acr...,https://www.seattletimes.com/seattle-news/poli...,0
4,Judge dismisses claims against 2 Seattle polic...,The Seattle Times,2019-01-01,"Lyles had a history of contacts with SPD, most...",https://www.seattletimes.com/seattle-news/crim...,0


In [405]:
text =df['link'][0].split('.')[1]
text

'seattletimes'

In [406]:
#Replace 'Others' in media with core website name taken from  link column
df[df['media']=='Others']=df[df['media']=='Others'].apply(lambda x : df['link'][0].split('.')[1]  )

In [407]:
np.sort(df.media.unique())

array(['(Burien) Blog', '10TV', '11Alive.com', '12News.com', '247Sports',
       '6ABC', '89.7 NPR News', '9News', 'ABA Journal', 'ABC 33/40',
       'ABC Chicago', 'ABC News', 'ABC10', 'ABC13', 'ABC4 Utah',
       'ABC7 San Francisco', 'ABS-CBN News', 'AP News', 'Action News Jax',
       'Action News Now', 'Akron Beacon Journal', 'Alaska Public Media',
       'AlterNet', 'American Civil Liberties Union',
       'American Security Today', 'Argus Leader', 'Arizona Daily Star',
       'Army.mil', 'Ars Technica', 'Atlas Obscura', 'Auburn Examiner',
       'Auburn Reporter', 'Austin American-Statesman', 'Axios', 'BBC',
       'Baltimore Sun', 'Bangor Daily News', 'Bay News 9',
       'Bellevue Reporter', 'Bellingham Herald', 'Bemidji Pioneer',
       'Billboard', 'Biometric Update', 'Blabbermouth.net',
       'Black Enterprise', 'Bloomberg Law', 'Bloomberg.com',
       'Boing Boing', 'Boise State Public Radio', 'Boston 25 News',
       'Boston Herald', 'Boston.com', 'Bothell-Kenmore Report

In [408]:
df[df['media']=='Others']

,title,media,date,desc,link,Local


In [409]:
#Checking for missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57051 entries, 0 to 57050
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   57051 non-null  object
 1   media   57051 non-null  object
 2   date    56770 non-null  object
 3   desc    57051 non-null  object
 4   link    57051 non-null  object
 5   Local   57051 non-null  object
dtypes: object(6)
memory usage: 2.6+ MB


In [410]:
#Dropping rows with missing date value
df=df.dropna(subset=['date'])

In [411]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56770 entries, 0 to 57050
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   56770 non-null  object
 1   media   56770 non-null  object
 2   date    56770 non-null  object
 3   desc    56770 non-null  object
 4   link    56770 non-null  object
 5   Local   56770 non-null  object
dtypes: object(6)
memory usage: 3.0+ MB


In [412]:
df=df[df['Local']==0]
df=df.drop_duplicates(subset=['title','desc','link','media'])
df.head()

,title,media,date,desc,link,Local
0,Seattle police release video of fatal shooting...,The Seattle Times,2019-01-01 00:00:00,"Faletogo died of a gunshot wound to the head, ...",https://www.seattletimes.com/seattle-news/crim...,0
2,Seattle Police release body cam footage in fat...,MyNorthwest.com,2019-01-01 00:00:00,The Seattle Police Department released the bod...,https://mynorthwest.com/1232749/seattle-police...,0
3,Former Seattle police chief Jim Pugel will run...,The Seattle Times,2019-01-01 00:00:00,More than two dozen candidates are running acr...,https://www.seattletimes.com/seattle-news/poli...,0
4,Judge dismisses claims against 2 Seattle polic...,The Seattle Times,2019-01-01 00:00:00,"Lyles had a history of contacts with SPD, most...",https://www.seattletimes.com/seattle-news/crim...,0
5,‘We have your DNA’: Seattle police create vide...,The Seattle Times,2019-01-01 00:00:00,We have your DNA. It is in your best interest ...,https://www.seattletimes.com/seattle-news/crim...,0


In [414]:
test_df=df
df.shape


(2891, 6)

In [282]:
media=df['media'][df['Local']==0].unique()
media

array(['The Seattle Times', 'MyNorthwest.com', 'KING 5', 'KIRO-TV',
       'Q13 FOX', 'Seattle Weekly', 'CHS Capitol Hill Seattle',
       'West Seattle Blog...', 'Seattle PI', 'Northwest Asian Weekly',
       'Real Change', 'KOMO', 'The Stranger', 'KOMO News',
       'Westside Seattle', 'West Seattle Blog…', 'International Examiner',
       'Seattle Spectator', 'Washington Times', 'South Seattle Emerald',
       'Seattle Times Interactives Projects', 'The Seattle Globalist',
       'dailyuw.com', 'Seattle Magazine |', 'Seattle Refined',
       'PubliCola', 'Seattle Met', 'The Seattle Medium'], dtype=object)

In [283]:
#All Media Sources
np.sort(df.media.unique())

array(['CHS Capitol Hill Seattle', 'International Examiner', 'KING 5',
       'KIRO-TV', 'KOMO', 'KOMO News', 'MyNorthwest.com',
       'Northwest Asian Weekly', 'PubliCola', 'Q13 FOX', 'Real Change',
       'Seattle Magazine |', 'Seattle Met', 'Seattle PI',
       'Seattle Refined', 'Seattle Spectator',
       'Seattle Times Interactives Projects', 'Seattle Weekly',
       'South Seattle Emerald', 'The Seattle Globalist',
       'The Seattle Medium', 'The Seattle Times', 'The Stranger',
       'Washington Times', 'West Seattle Blog...', 'West Seattle Blog…',
       'Westside Seattle', 'dailyuw.com'], dtype=object)

In [284]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

#Tokenize and Normalize (standardize) the text by removing all punctuation 
#from 'title' and 'desc' making all text lower case

df.title = df.title.apply(lambda x: nltk.RegexpTokenizer(r"\w+").tokenize(x.lower()))
df.desc = df.desc.apply(lambda x: nltk.RegexpTokenizer(r"\w+").tokenize(x.lower()))

In [285]:
#extend list of stopwords to include search terms
stopwords = nltk.corpus.stopwords.words('english')
newStopWords = ['seattle','police','officer','officers', 'shooting', 'killed', 'spd', 'department', 'said', 'say','call']
#'dependable','professional','respectful','delivering','quality','enforces','prevents']
stopwords.extend(newStopWords)

In [286]:
#Remove Stop words from the text and lemmetize/text stemming
#ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

'''df.title = df.title.apply(lambda x: [ps.stem(word) for word in x if word not in stopwords])
df.desc = df.desc.apply(lambda x: [ps.stem(word) for word in x if word not in stopwords])'''

df.title = df.title.apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in stopwords])
df.desc = df.desc.apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in stopwords])

In [287]:
#create trigram from tokenized list
from nltk.util import ngrams
def ngramconvert(df,n=3):
    for item in df[['title', 'desc']].columns:
        df['newtri'+item]=df[item].apply(lambda token_list: list(ngrams(token_list, n)))
    return df

In [288]:
ngramconvert(df, n=3)

#Initialize FreqDist objects
fdist_title = FreqDist()
fdist_desc = FreqDist()

#Measure of the frequency of words in the 'title' column
for lists_ in df['title']:
    for words in lists_:
        fdist_title[words]+=1
        
title_word_freq = pd.Series(fdist_title, name='freq_title').sort_values(ascending=False)

#Measure of the frequency of words in the 'desc' column
for lists_ in df['desc']:
    for words in lists_:
        fdist_desc[words]+=1
        
desc_word_freq = pd.Series(fdist_desc, name='freq_desc').sort_values(ascending=False)


In [289]:
#Calculating the top 50 words 
word_freq_table = pd.DataFrame(desc_word_freq).join(title_word_freq)

#word_freq_table['freq_total']= word_freq_table['freq_desc']+word_freq_table['freq_title']
word_freq_table = word_freq_table.fillna(int(0))
word_freq_table.head(20).sort_values(by=['freq_desc','freq_title'], ascending=False), word_freq_table.head(20).sort_values(by=['freq_title','freq_desc'], ascending=False)

(               freq_desc  freq_title
 city                 356       204.0
 year                 291       157.0
 according            261         2.0
 man                  224       392.0
 chief                170       110.0
 county               157        95.0
 council              150       125.0
 old                  148        86.0
 two                  142        47.0
 arrested             141       139.0
 one                  137        26.0
 new                  135       105.0
 people               133        49.0
 investigation        132        45.0
 report               125        55.0
 crime                124        85.0
 time                 123        27.0
 last                 118        12.0
 shot                 117       111.0
 suspect              111       158.0,
                freq_desc  freq_title
 man                  224       392.0
 city                 356       204.0
 suspect              111       158.0
 year                 291       157.0
 arrested  

In [290]:
#Trigram for news headlines
trigram_list =[]

for trigrams_ in df['newtrititle']:
    trigram_list.append(trigrams_)
trigram_headlines =pd.DataFrame(trigram_list)

In [291]:
trigram_headlines =trigram_headlines.stack()

In [292]:
trigram_headlines =pd.DataFrame(trigram_headlines)
trigram_headlines.columns = ['trigram_headlines']
trigram_headlines=trigram_headlines.reset_index().drop(['level_0', 'level_1'], axis=1)
trigram_headlines= trigram_headlines[trigram_headlines['trigram_headlines']!='None']

In [293]:
trigram_headlines.value_counts().head(30)

trigram_headlines                   
(mayor, jenny, durkan)                  20
(year, old, man)                        14
(cal, anderson, park)                   14
(west, crime, watch)                    14
(black, life, matter)                   12
(man, fatally, shot)                    11
(washington, state, nation)             11
(year, old, girl)                       11
(daily, news, update)                   10
(covid, 19, area)                       10
(chief, carmen, best)                   10
(coronavirus, daily, news)              10
(today, covid, 19)                      10
(area, washington, state)               10
(19, area, washington)                  10
(know, today, covid)                    10
(city, council, district)                8
(city, council, vote)                    8
(year, old, boy)                         8
(19, year, old)                          7
(fatally, shoot, man)                    7
(king, county, sheriff)                  6
(18, year, old)  

In [294]:
#Trigram for news article summaries
trigram_sumlist =[]

for trigrams_ in df['newtridesc']:
    trigram_sumlist.append(trigrams_)
trigram_summaries =pd.DataFrame(trigram_sumlist)

In [295]:
trigram_summaries =trigram_summaries.stack()

In [296]:
trigram_summaries =pd.DataFrame(trigram_summaries)
trigram_summaries.columns = ['trigram_news_summaries']
trigram_summaries=trigram_summaries.reset_index().drop(['level_0', 'level_1'], axis=1)
trigram_summaries= trigram_summaries[trigram_summaries['trigram_news_summaries']!='None']

In [297]:
trigram_summaries.value_counts().head(30)

trigram_news_summaries       
(mayor, jenny, durkan)           50
(chief, carmen, best)            35
(year, old, man)                 34
(chief, adrian, diaz)            29
(county, sheriff, office)        23
(interim, chief, adrian)         20
(black, life, matter)            19
(city, council, member)          18
(king, county, sheriff)          17
(line, 206, 233)                 15
(206, 233, 5000)                 15
(tip, line, 206)                 15
(harborview, medical, center)    14
(city, council, voted)           12
(photo, courtesy, layoff)        12
(courtesy, layoff, todd)         11
(layoff, todd, herman)           11
(7, 9, 2021)                     10
(19, year, old)                  10
(note, 7, 9)                     10
(herman, show, note)             10
(9, 2021, interim)               10
(todd, herman, show)             10
(cox, medium, group)             10
(washington, state, patrol)      10
(show, note, 7)                  10
(2021, interim, chief)            

An initial analysis of the trigram_summaries results (shown above) and the word frequency distribution table for the news sumarries indicates that there is a set a text that appears repeatedly for 2,254 times and it contains text. 

Upon further investigation, we were able to identify the source text on the Seattle Police Department's website:

"The Seattle Police Department (SPD) prevents crime, enforces laws, and supports quality public safety by delivering respectful, professional, and dependable police services. SPD operates within a framework that divides the city into five geographical areas called "precincts". These precincts define east, west, north, south, and southwest patrol areas, with a police station in each."

This means that 2,254 articles in the dataset are coming from SPD or from a https://www.seattle.gov/police site which have the above text at the bottom of those pages.)



In [298]:
#Save clean, preprocessed dataframe to csv
df.to_csv('news_clean_tokens.csv', index=False)

In [299]:
pip install wordcloud

Note: you may need to restart the kernel to use updated packages.


In [300]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [301]:

from wordcloud import WordCloud

In [ ]:
import numpy as np
from PIL import Image

# Create and generate a word cloud image:
spd_mask = np.array(Image.open('spd.png'))
wc = WordCloud(background_color='white',mask=spd_mask, max_words=50)

# for News summaries
wc.generate(str(df.desc))


# Display the generated image:
plt.figure(figsize = (16,9))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#For News Headlines
wc.generate(str(df.title))


# Display the generated image:
plt.figure(figsize = (16,9))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

## Topic Modeling 

In [304]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [305]:
#for LDA
import pyLDAvis


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [306]:
df_topic =df[['title', 'media', 'desc']]
df_topic.head()

,title,media,desc
0,"[release, video, fatal, new, year, eve]",The Seattle Times,"[faletogo, died, gunshot, wound, head, medical..."
2,"[release, body, cam, footage, fatal, aurora, a...",MyNorthwest.com,"[released, body, cam, footage, involving, car,..."
3,"[former, chief, jim, pugel, run, city, council]",The Seattle Times,"[two, dozen, candidate, running, across, seven..."
4,"[judge, dismisses, claim, 2, fatal, charleena,...",The Seattle Times,"[lyles, history, contact, call, report, domest..."
5,"[dna, create, video, get, killer, come, forward]",The Seattle Times,"[dna, best, interest, contact, u, immediately,..."


In [307]:
# Create Dictionary
headline_dictionary = corpora.Dictionary(df_topic.title)

# Create Corpus
headline_texts = df_topic['title']

# Term Document Frequency
corpus = [headline_dictionary.doc2bow(text) for text in headline_texts]


In [308]:
# Create Dictionary
summary_dictionary = corpora.Dictionary(df_topic.title)

# Create Corpus
summary_texts = df_topic['desc']

# Term Document Frequency
corpus_sum = [summary_dictionary.doc2bow(text) for text in summary_texts]


In [309]:
# number of topics
num_topics = 5

# Build LDA model for headlines
lda_model_headline = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=headline_dictionary,
                                       num_topics=num_topics,
                                       random_state =42)

In [310]:
# number of topics
num_topics = 5

# Build LDA model for summaries
lda_model_summary = gensim.models.LdaMulticore(corpus=corpus_sum,
                                       id2word=summary_dictionary,
                                       num_topics=num_topics,
                                       random_state =42)

In [311]:
from pprint import pprint

In [312]:
pprint(lda_model_headline.print_topics())
doc_lda_headline = lda_model_headline[corpus]

[(0,
  '0.016*"man" + 0.009*"chief" + 0.008*"city" + 0.007*"year" + 0.007*"arrest" '
  '+ 0.007*"video" + 0.006*"suspect" + 0.005*"death" + 0.005*"missing" + '
  '0.005*"council"'),
 (1,
  '0.016*"man" + 0.013*"woman" + 0.011*"shot" + 0.010*"city" + 0.007*"year" + '
  '0.005*"fatally" + 0.005*"death" + 0.005*"park" + 0.005*"precinct" + '
  '0.005*"video"'),
 (2,
  '0.014*"man" + 0.011*"city" + 0.011*"council" + 0.009*"2" + 0.006*"protest" '
  '+ 0.005*"new" + 0.005*"arrested" + 0.005*"dead" + 0.004*"say" + '
  '0.004*"mayor"'),
 (3,
  '0.021*"man" + 0.012*"arrested" + 0.010*"suspect" + 0.007*"west" + '
  '0.006*"protest" + 0.006*"year" + 0.005*"car" + 0.005*"fire" + 0.005*"city" '
  '+ 0.005*"5"'),
 (4,
  '0.008*"man" + 0.007*"year" + 0.006*"chief" + 0.006*"arrested" + '
  '0.006*"durkan" + 0.006*"mayor" + 0.006*"protest" + 0.005*"suspect" + '
  '0.005*"crime" + 0.005*"1"')]


In [313]:
pprint(lda_model_summary.print_topics())
doc_lda_summary = lda_model_summary[corpus_sum]

[(0,
  '0.010*"city" + 0.010*"year" + 0.010*"last" + 0.008*"chief" + 0.008*"man" + '
  '0.006*"according" + 0.006*"investigation" + 0.005*"video" + 0.005*"council" '
  '+ 0.004*"federal"'),
 (1,
  '0.015*"city" + 0.009*"shot" + 0.007*"year" + 0.006*"woman" + 0.006*"man" + '
  '0.006*"investigation" + 0.006*"park" + 0.006*"precinct" + 0.005*"east" + '
  '0.005*"wednesday"'),
 (2,
  '0.015*"city" + 0.011*"council" + 0.006*"people" + 0.006*"year" + '
  '0.005*"county" + 0.005*"two" + 0.005*"new" + 0.005*"man" + 0.005*"arrested" '
  '+ 0.005*"crime"'),
 (3,
  '0.015*"according" + 0.011*"year" + 0.010*"man" + 0.008*"arrested" + '
  '0.007*"old" + 0.007*"reported" + 0.006*"night" + 0.006*"one" + 0.006*"city" '
  '+ 0.005*"two"'),
 (4,
  '0.010*"year" + 0.010*"chief" + 0.009*"according" + 0.009*"city" + '
  '0.007*"mayor" + 0.007*"crime" + 0.006*"durkan" + 0.006*"old" + '
  '0.005*"county" + 0.005*"two"')]


In [314]:
from gensim.models import Word2Vec

In [315]:
cluster_model = Word2Vec(df['desc'], min_count=1)

In [316]:
cluster_model.wv.most_similar('asian', topn=10)

[('asked', 0.8727303743362427),
 ('state', 0.8665550351142883),
 ('east', 0.8660399913787842),
 ('information', 0.8654963970184326),
 ('part', 0.8646700382232666),
 ('car', 0.8644542694091797),
 ('report', 0.8643311858177185),
 ('patrol', 0.8640473484992981),
 ('shot', 0.8638507723808289),
 ('public', 0.8637387156486511)]

In [317]:
cluster_model.wv.most_similar('white', topn=10)  # - The word 'immigrant' is not present in the text

[('line', 0.7935531139373779),
 ('march', 0.793512761592865),
 ('2017', 0.7893218398094177),
 ('defund', 0.7868725061416626),
 ('000', 0.7865320444107056),
 ('50', 0.786282479763031),
 ('budget', 0.7857242822647095),
 ('got', 0.7855495810508728),
 ('another', 0.7848026752471924),
 ('patrol', 0.7844415307044983)]

In [318]:
cluster_model.wv.most_similar('african', topn=10)

[('assist', 0.5442954897880554),
 ('opa', 0.4687732458114624),
 ('pay', 0.46356847882270813),
 ('friend', 0.4597417414188385),
 ('tear', 0.45627257227897644),
 ('minneapolis', 0.45361268520355225),
 ('meeting', 0.4522327780723572),
 ('policing', 0.44754737615585327),
 ('please', 0.4461442530155182),
 ('dozen', 0.4417855143547058)]

In [319]:
cluster_model.wv.most_similar('spanish', topn=10)

[('coordinating', 0.3692871928215027),
 ('ronald', 0.35857197642326355),
 ('government', 0.3223722577095032),
 ('nicole', 0.32027468085289),
 ('administer', 0.3177845776081085),
 ('surviving', 0.3117476999759674),
 ('4333', 0.30967745184898376),
 ('hetle', 0.3064630627632141),
 ('8th', 0.3059508502483368),
 ('somewhere', 0.3052448630332947)]

## Vectorizing the data

In [344]:
#vectorize model
vectors_wv = cluster_model.wv.vectors
indicies_wv = {word:cluster_model.wv.get_vector(word, norm=True) for word in cluster_model.wv.key_to_index}

In [321]:
def vectorize(line): 
    words = []
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = indicies_wv[word]
        except KeyError: # if you do not have a vector for this word in your model, continue 
            continue
        words.append(vectors_wv[w2v_idx])
        if words: 
            words = np.asarray(words)
            min_vec = words.min(axis=0)
            max_vec = words.max(axis=0)
            return np.concatenate((min_vec, max_vec))
        if not words:
            return None 

In [322]:
X = [vectorize(line_) for line_ in df['desc']]
    

## KMeans clustering using scikit-learn

In [349]:
from sklearn import cluster, metrics

NUM_CLUSTERS=5

kmeans = cluster.KMeans(n_clusters=NUM_CLUSTERS)
kmeans.fit(vectors_wv)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [352]:
centroids[0][0:11]

array([-8.0469064e-04,  2.5542439e-03,  6.4649968e-04,  2.8309756e-04,
       -7.9932419e-05, -3.7139379e-03,  6.5906555e-04,  4.1457983e-03,
       -1.2015824e-03, -1.2586818e-03, -1.8448869e-03], dtype=float32)

In [394]:
0.16475916 in indicies_wv['man']

def get_key(val):
    for key_ in indicies_wv.keys():
        if val in indicies_wv[key_]:
            return print(key_)
    return print('not here')

get_it(-8.0469064e-04)

not here


## KMeans clustering using nltk

In [324]:
from nltk.cluster import KMeansClusterer, euclidean_distance
NUM_CLUSTERS=5
kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=euclidean_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors_wv, assign_clusters=True)


In [329]:
set(assigned_clusters) #test to confirm clustering

{0, 1, 2, 3, 4}

In [428]:

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time



# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')


# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)


# #############################################################################
# Load some categories from the training set

# Uncomment the following to do the analysis on all the categories
# categories = None



dataset = test_df

print("%d documents" % len(dataset.desc))

print()

labels = dataset.desc
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset "
      "using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words='english',
                                       alternate_sign=False, norm='l2')
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words='english',
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(dataset.desc)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=6, init='k-means++',
                          verbose=opts.verbose, random_state=42)
else:
    km = KMeans(n_clusters=6, init='k-means++', max_iter=200, 
                verbose=opts.verbose, random_state=42)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

df.title = df.title.apply(lambda x: [ps.stem(word) for word in x if word not in stopwords])
df.desc = df.desc.apply(lambda x: [ps.stem(word) for word in x if word not in stopwords])
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.
2891 documents

Extracting features from the training dataset using a sparse vectorizer
done in 0.381184s
n_samples: 2891, n_features: 3419

Clustering sparse data with MiniBatchKMeans(n_clusters=6, random_state=42, verbo

IndexError: index 6 is out of bounds for axis 0 with size 6